In [ ]:
%env GOOGLE_APPLICATION_CREDENTIALS=/tf/mounted/bl-scale_key.json
!head -n 3 $GOOGLE_APPLICATION_CREDENTIALS

In [ ]:
!pip3 install google-cloud-storage

In [ ]:
from google.cloud import storage
import os
from tqdm import tqdm
from multiprocessing import Pool, current_process
import time

In [ ]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # bucket_name = "your-bucket-name"
    # source_file_name = "local/path/to/file"
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)
    
def upload_dir(bucket_name, dir_name):
    start = time.time()
    paths = []
    for root, dirs, files in os.walk(os.path.join(dir_name, "filtered")):
        paths_temp = filter(lambda x: x.endswith(".png"), files)
        paths_temp = [os.path.join(root, path) for path in paths_temp]
        paths.extend(paths_temp)
        
    print(f"Found {len(paths)} stamps in {dir_name}, uploading")
        
    global upload_file
    def upload_file(path):
        upload_blob("bl-scale", path, "/".join(path.split("/")[-4:]))
        
    start_png = time.time()
    with Pool(os.cpu_count()) as p:
        p.map(upload_file, paths)
    del upload_file
        
    end_png = time.time()
    
    print(f"Uploaded {len(paths)} stamps in {end_png-start_png} seconds")
    print(f"Average upload time: {(end_png-start_png) / len(paths)}")
    
    print(f"Uploading metadata")
    
    upload_blob("bl-scale", os.path.join(dir_name, "header.pkl"),
                os.path.join(os.path.basename(dir_name), "header.pkl"))
    upload_blob("bl-scale", os.path.join(dir_name, "info_df.pkl"),
                os.path.join(os.path.basename(dir_name), "info_df.pkl"))
    
    end = time.time()
    
    print(f"Metadata uploaded")
    print(f"Outputs for {os.path.basename(dir_name)} uploaded in {end-start} seconds")
    

In [ ]:
!ls /tf/data/S-band-high/new/GBT_57578_74449_HIP34498_fine/

In [ ]:
paths = []
for root, dirs, files in tqdm(os.walk(os.path.join("/tf/data/S-band-high/", "GBT_58014_69579_HIP77629_fine/filtered"))):
    paths_temp = filter(lambda x: x.endswith(".png"), files)
    paths_temp = [os.path.join(root, path) for path in paths_temp]
    paths.extend(paths_temp)
print(len(paths))

In [ ]:
dir_name = os.path.join("/tf/data/S-band-high/new", "GBT_57578_74449_HIP34498_fine/")
for root, dirs, files in tqdm(os.walk(dir_name)):
    for file in files:
        print(os.path.join(os.path.dirname(dir_name).split("/")[-1], os.path.relpath(os.path.join(root, file), start=dir_name)))